In [1]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
import numba
import numpy as np
import numexpr as ne

from util import perf_comp_data
from native_util import load
%load_ext Cython
%load_ext version_information
%version_information numpy, Cython, numba, hope, numexpr

Software versions
Python 3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 6.1.0
OS Darwin 15.6.0 x86_64 i386 64bit
numpy 1.13.1
Cython 0.26
numba 0.34.0
hope 0.6.1
numexpr 2.6.2
Mon Sep 04 15:24:42 2017 CEST

In [2]:
# Python version

def ln_python(X, Y):
    Y[:] = (X-1) - (X-1)**2 / 2 + (X-1)**3 / 3 - (X-1)**4 / 4 + (X-1)**5 / 5 - (X-1)**6 / 6 + (X-1)**7 / 7 - (X-1)**8 / 8 + (X-1)**9 / 9


In [3]:
# Python version

def ln_python_exp(X, Y):
    x = (X - 1)
    x2 = x*x
    x4 = x2*x2
    x6 = x4*x2
    x8 = x4*x4
    Y[:] = x - x2 / 2 + x * x2 / 3 - x4 / 4 + x * x4 / 5 - x6 / 6 + x6 * x / 7 - x8 / 8 + x8 * x / 9

In [4]:
# NumExpr version
def ln_numexpr(X, Y):
    Y[:] = ne.evaluate("(X-1) - (X-1)**2 / 2 + (X-1)**3 / 3 - (X-1)**4 / 4 + (X-1)**5 / 5 - (X-1)**6 / 6 + (X-1)**7 / 7 - (X-1)**8 / 8 + (X-1)**9 / 9")


In [5]:
# Hope version
@hope.jit
def ln_hope(X, Y):
    Y[:] = (X-1) - (X-1)**2 / 2 + (X-1)**3 / 3 - (X-1)**4 / 4 + (X-1)**5 / 5 - (X-1)**6 / 6 + (X-1)**7 / 7 - (X-1)**8 / 8 + (X-1)**9 / 9


/Users/uweschmitt/Projects/hope/hope/jit.py:112: UserWarning: Recompiling... Reason: State is inconsistent with config. Inconsistent state key: [optimize].
  warnings.warn("Recompiling... Reason: {0}".format(le))


In [6]:
# Hope version

import hope
hope.config.optimize = False
@hope.jit
def ln_hope_exp(X, Y):
    x = (X - 1)
    x2 = x*x
    x4 = x2*x2
    x6 = x4*x2
    x8 = x4*x4
    Y[:] = x - x2 / 2 + x * x2 / 3 - x4 / 4 + x * x4 / 5 - x6 / 6 + x6 * x / 7 - x8 / 8 + x8 * x / 9

In [7]:
# numba version

@numba.jit
def ln_numba(X, Y):
    Y[:] = (X-1) - (X-1)**2 / 2 + (X-1)**3 / 3 - (X-1)**4 / 4 + (X-1)**5 / 5 - (X-1)**6 / 6 + (X-1)**7 / 7 - (X-1)**8 / 8 + (X-1)**9 / 9


In [8]:
# numba version

import numba

@numba.jit
def ln_numba_exp(X, Y):
    x = (X - 1)
    x2 = x*x
    x4 = x2*x2
    x6 = x4*x2
    x8 = x4*x4
    Y[:] = x - x2 / 2 + x * x2 / 3 - x4 / 4 + x * x4 / 5 - x6 / 6 + x6 * x / 7 - x8 / 8 + x8 * x / 9

In [9]:
%%cython

cimport cython

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def ln_cython(np.ndarray[np.double_t, ndim=1] X, np.ndarray[np.double_t, ndim=1] Y):
    cdef size_t i, n
    cdef double x
    n = len(X)
    cdef double *xx = <double *>X.data
    cdef double *yy = <double *>Y.data
    for i in range(n):
        x = xx[i] 
        yy[i] = (x-1) - (x-1)**2 / 2 + (x-1)**3 / 3 - (x-1)**4 / 4 + (x-1)**5 / 5 - (x-1)**6 / 6 + (x-1)**7 / 7 - (x-1)**8 / 8 + (x-1)**9 / 9

    
@cython.boundscheck(False)
@cython.wraparound(False)
def ln_cython_exp(np.ndarray[np.float_t, ndim=1] X, np.ndarray[np.float_t, ndim=1] Y):
    cdef size_t i, n
    cdef double x, x2, x4, x6, x8
    cdef double *xx = <double *>X.data
    cdef double *yy = <double *>Y.data
    n = len(X)
    for i in range(n):
        x = xx[i] - 1
        x2 = x * x
        x4 = x2 * x2
        x6 = x2 * x4
        x8 = x4 * x4
        yy[i] = x - x2 / 2 + x * x2 / 3 - x4 / 4 + x * x4 / 5 - x6 / 6 + x6 * x / 7 - x8 / 8 + x8 * x / 9    
    

In [10]:
from native_util import load
native_ln_mod = load("ln")
ln_native = native_ln_mod.run

native_ln_opt_mod = load("ln_opt")
ln_native_opt = native_ln_opt_mod.run

native_ln_exp_mod = load("ln_exp")
ln_native_exp = native_ln_exp_mod.run


running build_ext
building 'ln' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'
clang: ././src/ln.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -arch i386 -arch x86_64 -g ./src/ln.o -o ./ln.cpython-35m-darwin.so

running build_ext
building 'ln_opt' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/si

In [11]:
# Hope version - optimized

import hope
hope.config.optimize = True

@hope.jit
def ln_hope_opt(X, Y):
    Y[:] = (X-1) - (X-1)**2 / 2 + (X-1)**3 / 3 - (X-1)**4 / 4 + (X-1)**5 / 5 - (X-1)**6 / 6 + (X-1)**7 / 7 - (X-1)**8 / 8 + (X-1)**9 / 9


hope.config.optimize = False

/Users/uweschmitt/Projects/hope/hope/jit.py:112: UserWarning: Recompiling... Reason: State is inconsistent with config. Inconsistent state key: [optimize].
  warnings.warn("Recompiling... Reason: {0}".format(le))


In [12]:
import numpy as np

X = np.random.random(10000).astype(np.float64)
Y = np.ones_like(X)

In [13]:
Y1 = np.ones_like(X)
Y2 = np.ones_like(X)
Y3 = np.ones_like(X)
Y4 = np.ones_like(X)
Y5 = np.ones_like(X)
Y6 = np.ones_like(X)
Y7 = np.ones_like(X)
Y8 = np.ones_like(X)
Y9 = np.ones_like(X)
Y10 = np.ones_like(X)
Y11 = np.ones_like(X)
Y12 = np.ones_like(X)
Y13 = np.ones_like(X)

ln_python(X, Y1)
ln_python_exp(X, Y2)
ln_cython(X, Y3)
ln_cython_exp(X, Y4)
ln_numexpr(X, Y5)
ln_hope(X, Y6)
ln_hope_exp(X, Y7)
ln_numba(X, Y8)
ln_numba_exp(X, Y9)
ln_native(X, Y10)
ln_native_opt(X, Y11)
ln_native_exp(X, Y12)
ln_hope_opt(X, Y13)

assert np.allclose(Y1,Y2, 1E-10)
assert np.allclose(Y1,Y3, 1E-10)
assert np.allclose(Y1,Y4, 1E-10)
assert np.allclose(Y1,Y5, 1E-10)
assert np.allclose(Y1,Y6, 1E-10)
assert np.allclose(Y1,Y7, 1E-10)
assert np.allclose(Y1,Y8, 1E-10)
assert np.allclose(Y1,Y9, 1E-10)
assert np.allclose(Y1,Y10, 1E-10)
assert np.allclose(Y1,Y11, 1E-10)
assert np.allclose(Y1,Y12, 1E-10)
assert np.allclose(Y1,Y13, 1E-10)


Compiling following functions:
ln_hope(float64^1 X, float64^1 Y)
running build_ext
building 'ln_hope_f15670420910980a71b044ad212142e5c0bffbce3030fa3603b54df8_0' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'
clang: /var/folders/k8/zfp7dvcs1m326gz1brql1tv80000gn/T/hope8ltuei58/ln_hope_f15670420910980a71b044ad212142e5c0bffbce3030fa3603b54df8_0.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -arch i386 -arch x86_64 -g /var/folders/k8/zfp7dvcs1m326gz1brql1tv80000gn/T/hope8ltuei58/ln_hope_f15670420910980a71b044ad212142e5c0bffbce3030fa

In [14]:
import numexpr as ne

print("python")
%timeit ln_python(X, Y)
%timeit ln_python_exp(X, Y)
print("numexpr (1)")
ne.set_num_threads(1)
%timeit ln_numexpr(X, Y)
print("numexpr ({0})".format(ne.detect_number_of_cores()))
ne.set_num_threads(ne.detect_number_of_cores())
%timeit ln_numexpr(X, Y)
print("hope")
%timeit ln_hope(X, Y)
%timeit ln_hope_exp(X, Y)
%timeit ln_hope_opt(X, Y)
print("cython")
%timeit ln_cython(X, Y)
%timeit ln_cython_exp(X, Y)
print("numba")
%timeit ln_numba(X, Y)
%timeit ln_numba_exp(X, Y)
print("native")
%timeit ln_native(X, Y)
%timeit ln_native_opt(X, Y)
%timeit ln_native_exp(X, Y)


python
2.46 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
304 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
numexpr (1)
442 µs ± 7.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
numexpr (8)
211 µs ± 7.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
hope
2.37 ms ± 59.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
124 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.45 ms ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cython
2.24 ms ± 87.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
120 µs ± 3.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
numba
130 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
282 µs ± 3.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
native
2.4 ms ± 73.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
74 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
122 µs ±

In [15]:
from util import perf_comp_data

In [16]:
func_list = ["ln_python", "ln_python_exp", 
             "ln_numexpr", 
             "ln_hope", "ln_hope_exp", "ln_hope_opt", 
             "ln_cython", "ln_cython_exp", 
             "ln_numba", "ln_numba_exp", 
             "ln_native", "ln_native_opt", "ln_native_exp", ]
perf_comp_data(func_list,
               len(func_list)*["X, Y"], rep=100)

function: ln_native_opt       , av. time sec:   0.00006836, min. time sec:   0.00006819, relative:       1.0
function: ln_cython_exp       , av. time sec:   0.00010206, min. time sec:   0.00010199, relative:       1.5
function: ln_native_exp       , av. time sec:   0.00011374, min. time sec:   0.00011370, relative:       1.7
function: ln_numba            , av. time sec:   0.00011900, min. time sec:   0.00011891, relative:       1.7
function: ln_hope_exp         , av. time sec:   0.00013583, min. time sec:   0.00011382, relative:       2.0
function: ln_numexpr          , av. time sec:   0.00018525, min. time sec:   0.00017029, relative:       2.7
function: ln_numba_exp        , av. time sec:   0.00025554, min. time sec:   0.00025503, relative:       3.7
function: ln_python_exp       , av. time sec:   0.00028491, min. time sec:   0.00026110, relative:       4.2
function: ln_cython           , av. time sec:   0.00203083, min. time sec:   0.00191422, relative:      29.7
function: ln_hope_o